In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors
import numpy as np
#Import des librairies nécessaires

In [2]:
data = pd.read_csv(r"C:\Users\bapti\Desktop\WCS Python\Projet 2\movies_cleaned_ML.csv")
#Import de la donnée

In [3]:
data.info

<bound method DataFrame.info of       Unnamed: 0                  originalTitle ID_extract  startYear  \
0              0                Caroline chérie  tt0042316       1951   
1              1  Journal d'un curé de campagne  tt0042619       1951   
2              2                         Olivia  tt0042803       1951   
3              3                    Casque d'or  tt0043386       1952   
4              4                 Jeux interdits  tt0043686       1952   
...          ...                            ...        ...        ...   
4302        4302                         Police  tt9850344       2020   
4303        4303                       Boutchou  tt9861288       2020   
4304        4304                  Mon Ami Walid  tt9866344       2019   
4305        4305                       Felicità  tt9894450       2020   
4306        4306                        Le lion  tt9908390       2020   

      runtimeMinutes                          genres  averageRating  \
0                115

In [4]:
data = data[['originalTitle', 'startYear', 'ID_extract',
       'runtimeMinutes','genres', 'averageRating', 'Top_Box_office_$']]
#Filtre sur les colonnes souhaitées

In [5]:
data.head(5) #Affichage

,originalTitle,startYear,ID_extract,runtimeMinutes,genres,averageRating,Top_Box_office_$
0,Caroline chérie,1951,tt0042316,115,['Comedy'],5.9,0.0
1,Journal d'un curé de campagne,1951,tt0042619,95,['Drama'],7.8,47000.0
2,Olivia,1951,tt0042803,88,"['Comedy', 'Drama']",6.9,9934.0
3,Casque d'or,1952,tt0043386,94,"['Crime', 'Drama', 'Romance']",7.6,1413.0
4,Jeux interdits,1952,tt0043686,86,"['Comedy', 'Drama', 'War']",8.0,19889.0


In [7]:
data["runtimeMinutes"] = data["runtimeMinutes"].astype(int)
data["Top_Box_office_$"] = data["Top_Box_office_$"].astype(int)
data["startYear"] = data["startYear"].astype(int)
#conversion des données en entiers

C:\Users\bapti\AppData\Local\Temp\ipykernel_14608\158281857.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["runtimeMinutes"] = data["runtimeMinutes"].astype(int)


In [9]:
data["genres"] = data["genres"].str.replace("[","")
data["genres"] = data["genres"].str.replace("]","")
data["genres"].str.replace("'","")
data["list_genres"] = data["genres"].str.split(",")
data['genre1'] = data['list_genres'].str[0]
data['genre2'] = data['list_genres'].str[1]
data['genre3'] = data['list_genres'].str[2]
data["genre1"] = data["genre1"].str.replace("'","")
data["genre2"] = data["genre2"].str.replace("'","")
data["genre3"] = data["genre3"].str.replace("'","")
data[["genre1","genre2","genre3"]].head(5)
#création de 3 colonnes en fonction des genres du film

C:\Users\bapti\AppData\Local\Temp\ipykernel_14608\2126678470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["genres"] = data["genres"].str.replace("[","")


In [16]:
data = data.fillna("O")
data.head(5) #gestion des valeurs manquantes

In [18]:
# Création d'une colonne de catégorie de note
bins = [0, 4, 6, 7, 8, 10]
labels = ['Très mauvais', 'Mauvais', 'Moyen', 'Bon', 'Excellent']
data['categorie_rating'] = pd.cut(data['averageRating'], bins=bins, labels=labels)

In [19]:
data_ml = data[["ID_extract", "originalTitle",'startYear','averageRating', 'Top_Box_office_$','genre1','genre2','genre3','categorie_rating']].copy()
#copie des données filtrées et tranformées pour le machine learing

In [21]:
data.to_csv("data_movies_ML_final.csv") #export et sauvegarde des données filtrées
target = data_ml[["ID_extract","originalTitle"]] #données à prédire
data_ml = data_ml.drop("ID_extract", axis=1)
data_ml = data_ml.drop("originalTitle", axis=1) #suppression des colonnes à prédire

In [22]:
data_ml = pd.get_dummies(data_ml,prefix=['genre1', 'genre2', 'genre3','categorie_rating'], dtype=int)
# numérisation des données catégorielles

In [23]:
data_ml.head()

,startYear,averageRating,Top_Box_office_$,genre1_Action,genre1_Adventure,genre1_Animation,genre1_Biography,genre1_Comedy,genre1_Crime,genre1_Documentary,...,genre3_ Sport,genre3_ Thriller,genre3_ War,genre3_ Western,genre3_O,categorie_rating_Très mauvais,categorie_rating_Mauvais,categorie_rating_Moyen,categorie_rating_Bon,categorie_rating_Excellent
0,1951,5.9,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
1,1951,7.8,47000,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,1951,6.9,9934,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
3,1952,7.6,1413,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,1952,8.0,19889,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0


In [24]:
scaler = preprocessing.StandardScaler()
data_ml_norm = data_ml.copy()
for cols in data_ml_norm:
    data_ml_norm[cols] = scaler.fit_transform(np.array(data_ml_norm[cols]).reshape(-1, 1))

data_ml_norm.head()
#standardisation des données

,startYear,averageRating,Top_Box_office_$,genre1_Action,genre1_Adventure,genre1_Animation,genre1_Biography,genre1_Comedy,genre1_Crime,genre1_Documentary,...,genre3_ Sport,genre3_ Thriller,genre3_ War,genre3_ Western,genre3_O,categorie_rating_Très mauvais,categorie_rating_Mauvais,categorie_rating_Moyen,categorie_rating_Bon,categorie_rating_Excellent
0,-3.995793,-0.329238,-0.290112,-0.205183,-0.230339,-0.116834,-0.190612,1.311131,-0.243458,-0.258103,...,-0.077932,-0.205183,-0.111619,-0.026401,0.612861,-0.141036,1.298138,-0.888462,-0.430722,-0.102754
1,-3.995793,1.757714,-0.286075,-0.205183,-0.230339,-0.116834,-0.190612,-0.762700,-0.243458,-0.258103,...,-0.077932,-0.205183,-0.111619,-0.026401,0.612861,-0.141036,-0.770334,-0.888462,2.321682,-0.102754
2,-3.995793,0.769158,-0.289259,-0.205183,-0.230339,-0.116834,-0.190612,1.311131,-0.243458,-0.258103,...,-0.077932,-0.205183,-0.111619,-0.026401,0.612861,-0.141036,-0.770334,1.125541,-0.430722,-0.102754
3,-3.925024,1.538034,-0.289991,-0.205183,-0.230339,-0.116834,-0.190612,-0.762700,4.107477,-0.258103,...,-0.077932,-0.205183,-0.111619,-0.026401,-1.631691,-0.141036,-0.770334,-0.888462,2.321682,-0.102754
4,-3.925024,1.977393,-0.288404,-0.205183,-0.230339,-0.116834,-0.190612,1.311131,-0.243458,-0.258103,...,-0.077932,-0.205183,8.959026,-0.026401,-1.631691,-0.141036,-0.770334,-0.888462,2.321682,-0.102754


In [25]:
X = data_ml_norm[['startYear', 'averageRating', 'Top_Box_office_$', 'genre1_Action',
       'genre1_Adventure', 'genre1_Animation', 'genre1_Biography',
       'genre1_Comedy', 'genre1_Crime', 'genre1_Documentary', 'genre1_Drama',
       'genre1_Family', 'genre1_Fantasy', 'genre1_History', 'genre1_Horror',
       'genre1_Music', 'genre1_Musical', 'genre1_Mystery', 'genre1_Romance',
       'genre1_Sci-Fi', 'genre1_Thriller', 'genre1_Unknown', 'genre1_War',"categorie_rating_Bon",
       'categorie_rating_Excellent']]
# colonnes des données permettant de prédire le résultat

knn = NearestNeighbors(n_neighbors=5).fit(X)
distance, indice = knn.kneighbors() #instanciation des distances et index des voisins les plus proches

0   -0.102754
1   -0.102754
2   -0.102754
3   -0.102754
4   -0.102754
Name: categorie_rating_Excellent, dtype: float64
[[0.48351621 0.53747368 0.71616006 0.77375736 1.04530862]
 [0.17931964 0.5374809  0.56620568 0.67373943 0.69158401]
 [0.60727625 0.84639062 0.87717795 0.87833246 0.91651248]
 ...
 [0.09358932 0.10988304 0.11061905 0.11432111 0.11787242]
 [0.11144485 0.13204041 0.14159872 0.14215438 0.14325332]
 [0.14725519 0.14743459 0.15786967 0.17099447 0.17141764]] [[  24   34   70   55   39]
 [  12   29   45   51   32]
 [  39   24   76   34   55]
 ...
 [3889 1738 4114 4070 4219]
 [1968 4222 2440 1672 4132]
 [2256 2630 4218 2145 1794]]


In [26]:
l_idfilm = ["tt0046911"]
ids = []

for id in l_idfilm:
    id = data["originalTitle"][data["ID_extract"]==id]
    ids.append(id.index[0])
    print()

df_indice = pd.DataFrame(indice) #conversion de la liste des indices en DataFrame


[16]


In [37]:
print(f"Les voisins les plus proches de {data["originalTitle"].iloc[0]} sont :\n")
data.iloc[df_indice.iloc[0]] #Affichage des plus proches voisins


Les voisins les plus proches de Caroline chérie sont :



,originalTitle,startYear,ID_extract,runtimeMinutes,genres,averageRating,Top_Box_office_$,list_genres,genre1,genre2,genre3,categorie_rating
24,Elena et les hommes,1956,tt0049177,98,"'Comedy', 'Drama', 'Romance'",6.2,5568,"['Comedy', 'Drama', 'Romance']",Comedy,Drama,Romance,Moyen
34,Une parisienne,1957,tt0051133,86,'Comedy',6.2,0,['Comedy'],Comedy,O,O,Moyen
70,Madame Sans Gêne,1961,tt0056208,98,"'Comedy', 'Drama', 'Romance'",6.0,0,"['Comedy', 'Drama', 'Romance']",Comedy,Drama,Romance,Mauvais
55,Les jeux de l'amour,1960,tt0053964,86,'Comedy',6.3,4581,['Comedy'],Comedy,O,O,Moyen
39,La legge,1959,tt0051852,114,"'Comedy', 'Crime', 'Drama'",6.7,17351,"['Comedy', 'Crime', 'Drama']",Comedy,Crime,Drama,Moyen
